In [ ]:
!pip install pytube
!pip install opencv-python
!pip install torch
!pip install torchvision

In [1]:
from pytube import YouTube
import os
import glob
import cv2


file_dir = os.getcwd()
txt_list = glob.glob(os.path.join(file_dir, 'list.txt'))

In [ ]:
#영상 다운로더 1

for txt in txt_list:
    file_name = os.path.basename(txt)
    cls = file_name.split(".")[0]
    
    save_dir = os.path.join(file_dir,"src")
    
    if os.path.exists(save_dir) == False:
        os.mkdir(save_dir)

    print(f"Downloading videos is started")
    
    with open(txt, "r") as f:
        urllines = f.readlines()

        for idx ,url in enumerate(urllines):
            yt = YouTube(url)
            try:
                print(f"Video {yt.title} is downloading...")
                stream = yt.streams.filter(only_video=True).order_by('resolution').desc().first() #가장 화질 좋도록
                stream.download(save_dir)
            except:
                print(">>> Something is wrong, stop downloading current file...\n")
            
            
        print(f"< Downloading videos is finished >\n")

print("Done!")

In [1]:
#영상 다운로더 2 (하나하나 해야됨)

from pytube import YouTube
from pytube import Stream
from tqdm import tqdm

def progress_callback(stream: Stream, data_chunk: bytes, bytes_remaining: int) -> None:    
    pbar.update(len(data_chunk))


yt = YouTube('https://youtu.be/q-V4knoQ_4Y'           #여기다 url
             ,on_progress_callback=progress_callback)

yt_streams=yt.streams

for info, stream in enumerate(yt_streams):
    print(info,'영상 정보 ==>> ',stream)


itag = input('itag 숫자 선택해서 int 넣어주세요\n2160p(4k) 중에서 프레임 낮고, vcodec숫자 낮은걸로(hdr인데 용량차이), type=video로 : ')
# https://gist.github.com/sidneys/7095afe4da4ae58694d128b1034e01e2 에서 itag 숫자 참고해주세요. 2160p

my_stream = yt_streams.get_by_itag(itag)
print(my_stream,'다운합니다')
pbar = tqdm(total=my_stream.filesize,unit="bytes")

my_stream.download()
pbar.close()

print('done')

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
def extract_frame(file_dir ,file_name):
    
    filepath = os.path.join(file_dir,"src", file_name)
    video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함
    
    if not video.isOpened():
        print("Could not Open :", filepath)
        exit(0)


    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    print("length :", length)
    print("width :", width)
    print("height :", height)
    print("fps :", fps)
    
    frame_dir = os.path.join(file_dir,"frame")
    frame_path = os.path.join(frame_dir, file_name[:-4])
    
    try:
        if not os.path.exists(frame_path):
            os.makedirs(frame_path)
            
    except OSError:
        print ('Error: Creating directory. ' +  frame_path)

    count = 0
    success, image = video.read()
    print("Read a new frame : ", success)
        
    while success:
            print('Saved frame number :', count)
            cv2.imwrite(frame_path + "/frame%d.jpg" % count, image)
            video.set(cv2.CAP_PROP_POS_MSEC,(count*1000)) #1초 마다 추출
            success, image = video.read()
            print("Read a new frame : ", success)
            count += 1

    video.release()

In [ ]:
filepath = os.path.join(file_dir, "src")
file_list = []

for file in os.listdir(filepath):
    if file.endswith((".webm",".mp4")):
        file_list.append(file)

for file in file_list:
    extract_frame(file_dir, file)

In [2]:
import torch
import os
from shutil import copyfile,rmtree

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom
cwd = os.getcwd()
path = os.path.join(cwd, "frame")
dirs = os.listdir(path)

try:
    if os.path.exists(os.path.join(cwd,"runs")):
        yn = input("You've just done before. Do you want to remove 'runs' directory?\n(if not deleted, it will be reflected in the next result)\n(y/n)[n]: ")
        if yn.lower() == 'y':
            rmtree(os.path.join(cwd,"runs"))
            print("runs directory is just deleted!")
        else:
            print("it will be remained")
except OSError:
    print("Error: creating or removing directory")


for d in dirs:
        imgs = []

        # Images
        for file in os.listdir(os.path.join(path,d)):
            imgs.append(os.path.join(path,d, file))

        # Images
        for img in imgs:

        # Inference
                results = model(img)

        # Results
                crops = results.crop(save=True)  # or .show(), .save(), .print(), .pandas(), etc.

#copy croped image to summary directory

summary = os.path.join(cwd, "croped_images")

try:
    if not os.path.exists(summary):
        os.makedirs(summary)
    else:
        yn = input("Previous result directory exists. Do you want to remove? (y/n)[n]: ")
        if yn.lower() == 'y':
            rmtree(summary)
            os.makedirs(summary)
            print("you've just deleted previous result!")
        else:
            print("it wiil be remained")

except OSError:
    print("Error: Creating directory. " + summary)

from_dirs = os.path.join(cwd,"runs","detect")

idx = 1

for directory in os.listdir(from_dirs):
    car_crop_dir = os.path.join(from_dirs, directory, "crops","car")

    if not os.path.exists(car_crop_dir):
        continue

    for name in os.listdir(car_crop_dir):
        print("copying "+os.path.join(car_crop_dir, name)+"...")
        copyfile(os.path.join(car_crop_dir, name), os.path.join(summary,str(idx))+".jpg")
        idx+=1
print("done!")


Using cache found in C:\Users\h2hb0/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-10-24 Python-3.9.13 torch-1.12.1+cpu CPU

100.0%

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Saved 1 image to runs\detect\exp
Saved results to runs\detect\exp

Saved 1 image to runs\detect\exp2
Saved results to runs\detect\exp2

Saved 1 image to runs\detect\exp3
Saved results to runs\detect\exp3

Saved 1 image to runs\detect\exp4
Saved results to runs\detect\exp4

Saved 1 image to runs\detect\exp5
Saved results to runs\detect\exp5

Saved 1 image to runs\detect\exp6
Saved results to runs\detect\exp6

Saved 1 image to runs\detect\exp7
Saved results to runs\detect\exp7

Saved 1 image to runs\detect\exp8
Saved results to runs\detect\exp8

Saved 1 image to runs\detect\exp9
Saved results to runs\detect\exp9

Saved 1 image to runs\detect\exp10
Saved results to runs\detect\exp10

Saved 1 image to runs\detect\exp11
Saved results to runs\detect\exp1